# Contextual Word Sentiment Classification

This notebook implements a contextual word sentiment classification model using the IMDb dataset.
The goal is to classify individual words as positive, negative, or neutral based on sentence-level sentiment labels,
while incorporating the context of neighboring words.


**Important**: At the end you should write a report of adequate size, which will probably mean at least half a page. In the report you should describe how you approached the task. You should describe:
- Encountered difficulties (due to the method, e.g. "not enough training samples to converge", not technical like "I could not install a package over pip")
- Steps taken to alleviate difficulties
- General description of what you did, explain how you understood the task and what you did to solve it in general language, no code.
- Potential limitations of your approach, what could be issues, how could this be hard on different data or with slightly different conditions
- If you have an idea how this could be extended in an interesting way, describe it.


In [2]:
# Required Libraries
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from nltk.tokenize import word_tokenize
from collections import defaultdict
from tqdm import tqdm
import nltk

nltk.download('punkt_tab')  # For tokenization

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [3]:
# !pip install 'portalocker>=2.0.0'

In [4]:
# Task 1: Load and preprocess the IMDb dataset
# IMDb dataset can be loaded from torchtext or manually via pandas
from torchtext.datasets import IMDB

# Load IMDb dataset
train_data, test_data = IMDB(split=('train', 'test'))

# Convert the data into lists
train_data = [(label, text) for label, text in train_data]
test_data = [(label, text) for label, text in test_data]

# Combine the datasets for label propagation
data = train_data + test_data

# Convert to DataFrame for easier processing
df = pd.DataFrame(data, columns=['label', 'text'])

ModuleNotFoundError: No module named 'torchtext'

In [ ]:
pd.set_option('display.max_colwidth', None)
df.groupby('label').sample(5)

In [ ]:
## Tip. You can get the dataset from torchtext but the package is old and needs pytorch version 2.2 to work
## If you want to use it choose your versions like this:
!pip install -U torch==2.2.0 torchvision==0.17.0 torchaudio==2.2.0 --index-url https://download.pytorch.org/whl/cu121 torchtext
# from torchtext.datasets import IMDB

Looking in indexes: https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 757.3/757.3 MB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 72.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 68.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 53.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 69.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 40.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 84.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.

In [ ]:
# Task 2: Implement tokenization and label propagation
# Implement a function to calculate sentiment scores for each word based on sentence-level labels.
# The function should propagate labels to individual words and calculate a soft score for each word.

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

nltk.download('stopwords')
nltk.download('wordnet')

In [ ]:
nltk.download('averaged_perceptron_tagger')

def get_wordnet_pos(tag):
    """Convertit les tags POS de nltk en tags WordNet."""
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN  # Par défaut, on retourne 'NOUN'

In [ ]:
def preprocess_text(text):

  tokens = word_tokenize(text.lower())
  tokens = [token.lower() for token in tokens if token.isalpha()]

  tokens = [token for token in tokens if token not in stopwords.words('english') and token != 'br']

  lemmatizer = nltk.stem.WordNetLemmatizer()
  pos_tags = nltk.pos_tag(tokens)
  tokens = [lemmatizer.lemmatize(token, get_wordnet_pos(tag)) for token, tag in pos_tags]

  return tokens

df['tokens'] = df['text'].apply(preprocess_text)
df.head()

In [ ]:
df.to_csv('data_preprocessed.csv')

In [ ]:
# Hint: You can use word_tokenize for tokenization
# Hint: You can use a dictionary to store counts of positive and negative labels for each word.

# Task 3: Prepare data for contextual learning
# Implement a class to create a dataset with context windows.
# Each data point should include the word embedding for the target word,
# as well as an averaged embedding of the context words in a defined window size.

# Use a pre-trained embedding model like GloVe. Download the embeddings and load them into a dictionary.
# Example: {"word": embedding_vector}

# Class signature example:
# class WordContextDataset(Dataset):
#     def __init__(self, df, word_scores, embedding_model, window_size=2):
#         # Your code here
#         pass

#     def __len__(self):
#         # Your code here
#         pass

#     def __getitem__(self, idx):
#         # Your code here
#         pass

In [ ]:
# Task 4: Define and train the model
# Define a neural network for sentiment classification using PyTorch.
# The network should take an input vector of concatenated word and context embeddings.

In [ ]:
# Example:
# class SentimentClassifier(nn.Module):
#     def __init__(self, input_dim):
#         super(SentimentClassifier, self).__init__()
#         # Your code here

#     def forward(self, x):
#         # Your code here
#         pass

# Implement a training loop to train the model on the dataset created.

In [ ]:
# Task 5: Evaluate the model
# Evaluate the trained model on a validation set.
# Use metrics such as precision, recall, and F1-score.

In [ ]:
# Example code to evaluate the model:
# with torch.no_grad():
#     # Predict on validation data and calculate metrics
#     pass

# Optional: Experiment with hyperparameters or model architecture to improve performance.
# Examples: Try different window sizes, embedding dimensions, or additional layers in the model.